# Query Fee Rate

In [1]:
# standard imports
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
from eth_utils import keccak
#from web3 import Web3

### Ford Subgraph

In [2]:
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

In [3]:
# function used to query each type of event from The Graph
def query_theGraphFord(raw_query, field_name, url, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}"
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + indexer + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [4]:
# get allocationCollected event
collect_query = """query{
  allocationCollecteds() {
    id
    blockNumber
    logIndex
    indexer
    subgraphDeploymentID
    epoch
    tokens
    allocationID
    from
    curationFees
    rebateFees
  }
  
}"""

field_name = 'allocationCollecteds'
alloc_collected_df = query_theGraphFord(collect_query, field_name, url, False)

# save allocationCollected data
alloc_collected_df['type'] = 'allocationCollecteds'
alloc_collected_df.to_csv('allocationCollected.csv', index = False, header = True)

In [21]:
alloc_collected_df.head()

,id,blockNumber,logIndex,indexer,subgraphDeploymentID,epoch,tokens,allocationID,from,curationFees,rebateFees,type
0,0x4bd3585834cb6bf0477d9e45f49d075d9cb50e780107...,11515164,306,0x362717d3fa1bfe457535b9b716f450b95a858217,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,10,1625600000000000000,0x69b8a4c937638654c6d14340602d94b9627d4c19,0x9fd8ac559219d4dd463193d1fb0433f7844f730f,160934400000000000,1448409600000000000,allocationCollecteds
1,0xff642404c02f51e4530e00e3450e802edb738aeb50a5...,11515417,318,0xd11e05240a50e2eedb03e72d4c9c6465bc05f2a4,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,10,162645000000000000,0x322fd9ad3b2f0498a4da371bdc499224b70dd3b8,0x9fd8ac559219d4dd463193d1fb0433f7844f730f,16101855000000000,144916695000000000,allocationCollecteds
2,0x019bf5e9219ca09d3e981099f396859bc1bd8cf9d8d8...,11515434,161,0xd11e05240a50e2eedb03e72d4c9c6465bc05f2a4,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,10,162645000000000000,0x322fd9ad3b2f0498a4da371bdc499224b70dd3b8,0x9fd8ac559219d4dd463193d1fb0433f7844f730f,16101855000000000,144916695000000000,allocationCollecteds
3,0x207a27ec6f037f62d03e9a9246a5a2b5216c4b55efba...,11515434,167,0xc430be492ddeb6e761dbbd0e08bafe99f5064d90,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,10,3545973625650624567,0x02353e9c1d14fe8e3eccf316fe6dde4aaf43cf58,0x9fd8ac559219d4dd463193d1fb0433f7844f730f,351051388939411832,3159462500454706490,allocationCollecteds
4,0x4990bb5078ab860f857a3e6fab2a800e8cdb60d043a4...,11515434,149,0xd11e05240a50e2eedb03e72d4c9c6465bc05f2a4,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,10,162645000000000000,0x322fd9ad3b2f0498a4da371bdc499224b70dd3b8,0x9fd8ac559219d4dd463193d1fb0433f7844f730f,16101855000000000,144916695000000000,allocationCollecteds


In [42]:
# Handling Rebate Fees that cuased Value Error

for count, i in enumerate(alloc_collected_df.rebateFees):
    # print(i)
    
    try: 
        pd.to_numeric(alloc_collected_df.rebateFees, downcast='float')
    except ValueError:
        alloc_collected_df.rebateFees.iloc[count] = float(i)
    

In [35]:
alloc_time_series = alloc_collected_df.sort_values(by=['subgraphDeploymentID','blockNumber'])

In [36]:
alloc_time_series.to_csv('alloc_time_series.csv', index = False, header = True)

In [43]:
# Sum of Fees over Epoch
alloc_ts_epoch = alloc_time_series.groupby(by=['subgraphDeploymentID','epoch']).sum()

In [44]:
alloc_ts_epoch.head()

id  \
subgraphDeploymentID                               epoch                                                      
0x008f49562d4bdb43ae1b4b68097952d174fcec525019b... 227    0x1a004713b95fe2d54c54591f7717643b8ff2e1f9ed1a...   
                                                   242    0xb096ef3066b8ed8d144d12a46150b8f8c6eb3c165199...   
0x014e8a3184d5fad198123419a7b54d5f7c9f8a9811164... 162    0x0e2daa85803d2724b1dec802975ca3b67a5597bdc907...   
                                                   163    0xf9190efd77db5b0217872a9bf3d20409d634a8f77388...   
                                                   164    0x06faf211045153ff1d0783e3057c1bcc2db888977035...   

                                                               blockNumber  \
subgraphDeploymentID                               epoch                     
0x008f49562d4bdb43ae1b4b68097952d174fcec525019b... 227            12955969   
                                                   242            13056023   
0x014e8a3184d5fad198123419a7b54d5f7c9f8a9811164... 162            12525387   
                                                   163    1253018512534875   
                                                   164    1254120812541221   

                                                         logIndex  \
subgraphDeploymentID                               epoch            
0x008f49562d4bdb43ae1b4b68097952d174fcec525019b... 227        213   
                                                   242         99   
0x014e8a3184d5fad198123419a7b54d5f7c9f8a9811164... 162        200   
                                                   163     338172   
                                                   164     315157   

                                                                                                    indexer  \
subgraphDeploymentID                               epoch                                                      
0x008f49562d4bdb43ae1b4b68097952d174fcec525019b... 227           0x453b5e165cf98ff60167ccd3560ebf8d436ca86c   
                                                   242           0x4437e367cc68d87675148a9e4a1f052f0359c880   
0x014e8a3184d5fad198123419a7b54d5f7c9f8a9811164... 162           0xc60d0c8c74b5d3a33ed51c007ebae682490de261   
                                                   163    0x5a8904be09625965d9aec4bffd30d853438a053e0x5a...   
                                                   164    0xbb784d9b398271b7a64f975bebde8694096919150x92...   

                                                                                               tokens  \
subgraphDeploymentID                               epoch                                                
0x008f49562d4bdb43ae1b4b68097952d174fcec525019b... 227                                 20000000000000   
                                                   242                                180000000000000   
0x014e8a3184d5fad198123419a7b54d5f7c9f8a9811164... 162                           22877840000000000000   
                                                   163     507108200000000000000886653900000000000000   
                                                   164    4291518469373966469591413539620108275315942   

                                                                                               allocationID  \
subgraphDeploymentID                               epoch                                                      
0x008f49562d4bdb43ae1b4b68097952d174fcec525019b... 227           0x86bd496d39756efc8a9f785b9e613a8c732a48ed   
                                                   242           0x2fc428d4a7fa2b2f0f5077566f7c42618535a04d   
0x014e8a3184d5fad198123419a7b54d5f7c9f8a9811164... 162           0xd1e26b108ceecebaf5ef759d925f76ad178ef7f2   
                                                   163    0x40a45c6074cf6b32c6adc889e733aa5b4eb8ee6c0x3e...   
                                                   164    0xbcc9855097cb35b2121ca0ae27ff8d0275f8a7570xe8...   

    

In [45]:
alloc_ts_epoch.to_csv('alloc_ts_epoch.csv', index = False, header = True)

In [31]:
alloc_collected_df.rebateFees.iloc[124] = float(alloc_collected_df.rebateFees.iloc[124])
alloc_collected_df.rebateFees.iloc[145] = float(alloc_collected_df.rebateFees.iloc[145])

In [5]:
# get rewardsAssigned data
rewardsassigned_query = """query{rewardsAssigneds () {
    id
    blockNumber
    logIndex
    indexer
    amount
    epoch
}
}"""
field_name = 'rewardsAssigneds'
rewards_assigned_df = query_theGraphFord(rewardsassigned_query, field_name, url, False)

# save rewardsAssigned data
rewards_assigned_df['type'] = 'rewardsAssigneds'
rewards_assigned_df.to_csv('rewardsAssigneds.csv', index = False, header = True)

In [6]:
# get rebateClaim data
rebate_query = """query{rebateClaimeds () {
    id
    blockNumber
    logIndex
    indexer
    tokens
    subgraphDeploymentID
    allocationID
    delegationFees
    epoch
}
}"""
field_name = 'rebateClaimeds'
rebate_claim_df = query_theGraphFord(rebate_query, field_name, url, False)

# save rebateClaim data
rebate_claim_df['type'] = 'rebateClaimeds'
rebate_claim_df.to_csv('rebateClaimeds.csv', index = False, header = True)

### Mainnet Subgraph

In [7]:
# function used to query each type of event from The Graph
def query_theGraph(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0, type:' + trans_type +'})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + str(indexer) + ', type:' + trans_type +'})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['blockNumber']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

### Subgraph Query Function

In [8]:
# function used to query each type of event from The Graph
def query_theGraph_subgraph(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:createdAt, where:{createdAt_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:createdAt, where: {createdAt_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [9]:
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

In [10]:

subgraph_query = '''query{
	subgraphs(){
    id
    owner{
      id
    }
    currentVersion{
      id
      subgraphDeployment{
        id
      }
      version
    }
    createdAt
    updatedAt
    active
    migrated
    displayName
  }
}
'''

field_name = 'subgraphs'
trans_type = 'BurnNSignal'

subgraph_df = query_theGraph_subgraph(subgraph_query, field_name, url, trans_type, False)

In [13]:
subgraph_df.head()

,id,owner,currentVersion,createdAt,updatedAt,active,migrated,displayName
0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0,{'id': '0x93606b27cb5e4c780883ec4f6b7bed5f6572...,{'id': '0x93606b27cb5e4c780883ec4f6b7bed5f6572...,1608252009,1645807948,False,True,PoolTogether
1,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,{'id': '0x93606b27cb5e4c780883ec4f6b7bed5f6572...,{'id': 'ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasU...,1608252009,1645807948,False,True,PoolTogether
2,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,1619441050,1619790992,False,False,PoolTogether
3,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,{'id': 'Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1...,1619441050,1619790992,False,False,PoolTogether
4,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,{'id': '0x673b6e9fe607f6ddf4a4f25b386b846c5c82...,1619442230,1645807948,False,True,PoolTogether


In [14]:
subgraph_df['ownerId'] = [i['id'] for i in subgraph_df['owner']]
subgraph_df['version'] = [i['version'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphDeploymentId'] = [i['subgraphDeployment']['id'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphid'] = [i['id'] for i in subgraph_df['currentVersion']]
subgraph_df.drop(['owner', 'currentVersion'], axis = 1, inplace = True)
subgraph_df.head()

,id,createdAt,updatedAt,active,migrated,displayName,ownerId,version,subgraphDeploymentId,subgraphid
0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0,1608252009,1645807948,False,True,PoolTogether,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0-0
1,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,1608252009,1645807948,False,True,PoolTogether,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN-0
2,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0,1619441050,1619790992,False,False,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0-0
3,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4,1619441050,1619790992,False,False,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4-0
4,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1,1619442230,1645807948,False,True,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0xbbde25a2c85f55b53b7698b9476610c3d1202d88870e...,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1-0


In [15]:
subgraph_df['subgraph_version'] = [i.split('-')[0]+'-'+i.split('-')[1] for i in  subgraph_df['subgraphid']]
subgraph_df.head()

,id,createdAt,updatedAt,active,migrated,displayName,ownerId,version,subgraphDeploymentId,subgraphid,subgraph_version
0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0,1608252009,1645807948,False,True,PoolTogether,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0-0,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd-0
1,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN,1608252009,1645807948,False,True,PoolTogether,0x93606b27cb5e4c780883ec4f6b7bed5f6572d1dd,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN-0,ziL9z9yVvVoksZ5MhyQFNjtspRdxMNDvwtYasUtXCvN-0
2,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0,1619441050,1619790992,False,False,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0-0,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-0
3,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4,1619441050,1619790992,False,False,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0x31edcacc9a53bc8ab4be2eeb0d873409da4c4228cb2d...,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4-0,Gv1rewdMuEeDQ9sWsWMjJfwKWrTxo5e19vgxr1br7Ut4-0
4,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1,1619442230,1645807948,False,True,PoolTogether,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e,0,0xbbde25a2c85f55b53b7698b9476610c3d1202d88870e...,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1-0,0x673b6e9fe607f6ddf4a4f25b386b846c5c82995e-1


In [17]:
# create list of subgraphs indexer is involved in
unique_subgraphs = subgraph_df['subgraphDeploymentId'].unique()

In [20]:
len(unique_subgraphs)

352

# GNS Exploration

In [ ]:
#get MintNSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

name_mint_query = '''query{
  nameSignalTransactions(){
    id
    blockNumber
    signer{
      curator{
        id
      }
    }
    type
    nameSignal
    versionSignal
    tokens
    subgraph{
      id
      currentVersion
      {
        version
        subgraphDeployment {
          id
        }
      }
      displayName
    }
  }
}
'''

field_name = 'nameSignalTransactions'
trans_type = 'MintNSignal'

name_mint_df = query_theGraph(name_mint_query, field_name, url, trans_type, False)

In [ ]:
name_mint_df.head()

In [ ]:
name_mint_df['curatorId'] = [i['curator']['id'] for i in name_mint_df['signer']]
name_mint_df['subgraphId'] = [i['id'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphDeploymentID'] = [i['currentVersion']['subgraphDeployment']['id'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphversion'] = [i['currentVersion']['version'] for i in name_mint_df['subgraph']]
name_mint_df['subgraphname'] = [i['displayName'] for i in name_mint_df['subgraph']]
name_mint_df.drop(['signer','subgraph'], axis = 1, inplace = True)
name_mint_df.head()

In [ ]:
#get BurnNSignal event
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

name_burn_query = '''query{
  nameSignalTransactions(){
    id
    blockNumber
    signer{
      curator{
        id
      }
    }
    type
    nameSignal
    versionSignal
    tokens
    subgraph{
      id
      currentVersion
      {
        version
        subgraphDeployment {
          id
        }
      }
      displayName
    }
  }
}
'''

field_name = 'nameSignalTransactions'
trans_type = 'BurnNSignal'

name_burn_df = query_theGraph(name_burn_query, field_name, url, trans_type, False)

In [ ]:
name_burn_df.head()

In [ ]:
name_burn_df['curatorId'] = [i['curator']['id'] for i in name_burn_df['signer']]
name_burn_df['subgraphId'] = [i['id'] for i in name_burn_df['subgraph']]
name_burn_df['subgraphDeploymentID'] = [i['currentVersion']['subgraphDeployment']['id'] for i in name_burn_df['subgraph']]
name_burn_df['subgraphversion'] = [i['currentVersion']['version'] for i in name_burn_df['subgraph']]
name_burn_df['subgraphname'] = [i['displayName'] for i in name_burn_df['subgraph']]
name_burn_df.drop(['signer','subgraph'], axis = 1, inplace = True)
name_burn_df.head()

In [ ]:
indexer_name_curation_df = name_mint_df[name_mint_df['subgraphDeploymentID'].isin(unique_subgraphs)].append(name_burn_df[name_burn_df['subgraphDeploymentID'].isin(unique_subgraphs)])

In [ ]:
#merge curation events
single_indexer_name_df = indexer_df.append(indexer_name_curation_df)
single_indexer_name_df.sort_values(by='blockNumber')
single_indexer_name_df.head()

In [ ]:
# save all events for the indexer
single_indexer_name_df.to_csv('../another_indexer/single_indexer/singleIndexername.csv', index = False, header = True)

In [ ]:
# function used to query each type of event from The Graph
def query_theGraph_subgraph(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:createdAt, where:{createdAt_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:createdAt, where: {createdAt_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [ ]:
#get subgraphs data
url = 'https://api.thegraph.com/subgraphs/name/graphprotocol/graph-network-mainnet'

subgraph_query = '''query{
	subgraphs(){
    id
    owner{
      id
    }
    currentVersion{
      id
      subgraphDeployment{
        id
      }
      version
    }
    createdAt
    updatedAt
    active
    migrated
    displayName
  }
}
'''

field_name = 'subgraphs'
trans_type = 'BurnNSignal'

subgraph_df = query_theGraph_subgraph(subgraph_query, field_name, url, trans_type, False)

In [ ]:
subgraph_df.head()

In [ ]:
subgraph_df['ownerId'] = [i['id'] for i in subgraph_df['owner']]
subgraph_df['version'] = [i['version'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphDeploymentId'] = [i['subgraphDeployment']['id'] for i in subgraph_df['currentVersion']]
subgraph_df['subgraphid'] = [i['id'] for i in subgraph_df['currentVersion']]
subgraph_df.drop(['owner', 'currentVersion'], axis = 1, inplace = True)
subgraph_df.head()

In [ ]:
subgraph_df[10:20]

In [ ]:
subgraph_df['subgraph_version'] = [i.split('-')[0]+'-'+i.split('-')[1] for i in  subgraph_df['subgraphid']]
subgraph_df.head()

In [ ]:
# function used to query each type of event from The Graph
def query_theGraph_events(raw_query, field_name, url, trans_type, verbose=False):

    query_parts =raw_query.split(')')
    paginator = ", where: {}" 
    n = 0
    records = []
    while n < 1000:
        if (n == 0):
            # order by blockNumber to get all events, as deafult ordering is by id
            query = 'query '+query_parts[0] + 'orderBy:blockNumber, where:{blockNumber_gt: 0})' + query_parts[1]
        else:
            query = 'query '+query_parts[0]+ 'orderBy:blockNumber, where: {blockNumber_gt: ' + str(indexer) + '})'+query_parts[1]

        if verbose:
            print(query)

        r = requests.post(url, json = {'query':query})

        try:
            d = json.loads(r.content)['data'][field_name]
        except:
            print(r.content)
            errors = json.loads(r.content)['errors']
            print(errors)
            for e in errors:
                print(e['message'])

        #print(f'results {len(d)}')
        records.extend(d)
        #print(f'total {len(records)}')
        try:
            indexer = d[-1]['createdAt']
        except:
            print('noblock')
        
        n += 1
        if len(d) < 99:
            break
        
    return pd.DataFrame(records)

In [ ]:
#get grtWithdrawns event
url = 'https://api.thegraph.com/subgraphs/name/fordn/graph-network-events'

withdraw_query = '''query{
	grtwithdrawns(){
        id
        blockNumber
        graphAccount
        subgraphNumber
        nameCurator
        nSignalBurnt
        withdrawnGRT
  }
}
'''

field_name = 'grtwithdrawns'

withdraw_df = query_theGraph_events(withdraw_query, field_name, url, trans_type, False)

In [ ]:
withdraw_df.head()

In [ ]:
withdraw_df['subgraph_w_version'] = withdraw_df['graphAccount']+'-'+withdraw_df['subgraphNumber']
withdraw_df.head()

In [ ]:
withdraw_df['subgraphDeploymentID'] = 0

In [ ]:
for i, j in enumerate(withdraw_df['subgraph_w_version']):
    for k, l in enumerate(subgraph_df['subgraph_version']):
        if j in l:
            withdraw_df['subgraphDeploymentID'][i] = subgraph_df.iloc[k]['subgraphDeploymentId']
withdraw_df['type'] = 'grtwithdrawns'
withdraw_df.head()

In [ ]:
withdraw_df[withdraw_df['subgraphDeploymentID'].isin(unique_subgraphs)]

In [ ]:
single_indexer_name_df = single_indexer_name_df.append(withdraw_df[withdraw_df['subgraphDeploymentID'].isin(unique_subgraphs)])
single_indexer_name_df['blockNumber'] = single_indexer_name_df['blockNumber'].astype(int)
single_indexer_name_df.drop(['index'], axis = 1, inplace = True)
single_indexer_name_df.reset_index(inplace = True)
single_indexer_name_df['epoch'] = ((single_indexer_name_df['blockNumber'] - 11446768)/6646)
single_indexer_name_df['epoch'] = single_indexer_name_df['epoch'].astype(int)
single_indexer_name_df.sort_values(by=['blockNumber', 'epoch'], inplace = True)
single_indexer_name_df.head()

In [ ]:
single_indexer_name_df.columns

In [ ]:
single_indexer_name_df['type'].unique()

In [ ]:
# save all events for the indexer
single_indexer_name_df.to_csv('../another_indexer/single_indexer/singleIndexername.csv', index = False, header = True)

In [ ]:
single_indexer_name_df[['tokens', 'type']]

In [ ]:
curation_list = []
signal_list = []
epoch_list = []
for i, j in single_indexer_name_df[['tokens', 'type', 'nameSignal', 'blockNumber', 'epoch']].iterrows():
    #print(j['epoch'])
    if j['type'] == 'MintNSignal':
        curation_list.append(int(j['tokens']))
        signal_list.append(int(j['nameSignal']))
        epoch_list.append(int(j['epoch']))
    elif j['type'] == 'BurnNSignal':
        curation_list.append(-int(j['tokens']))
        signal_list.append(-int(j['nameSignal']))
        epoch_list.append(int(j['epoch']))
    else:
        curation_list.append(0)
        signal_list.append(0)
        epoch_list.append(int(j['epoch']))
curation_list = np.cumsum(curation_list)
signal_list = np.cumsum(signal_list)
plt.figure(figsize = (10, 5))
plt.plot(epoch_list , curation_list)
plt.xlabel('Epoch')
plt.ylabel('Tokens')
#plt.ylim(1e4, 1e6)
plt.title('Curation Tokens on Subgraphs Single Indexer is In')

In [ ]:
delegation_list = []
for i, j in single_indexer_name_df[['tokens', 'type', 'blockNumber']].iterrows():
    if j['type'] == 'stakeDelegateds':
        delegation_list.append(int(j['tokens']))
    elif j['type'] == 'stakeDelegatedWithdrawns':
        delegation_list.append(-int(j['tokens']))
    else:
        delegation_list.append(0)
delegation_list = np.cumsum(delegation_list)


In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(single_indexer_name_df.epoch, signal_list)
plt.xlabel('Timestep')
plt.ylabel('Tokens')
#plt.ylim(1e4, 1e6)
plt.title('Amount of Signal Curators Have for Subgraphs Single Indexer is In')

In [ ]:
token_list = ['Curated Tokens', 'Delegated Tokens']
total = {}
total['Curated Tokens'] = np.array(curation_list, dtype=float)
total['Delegated Tokens'] = np.array(delegation_list, dtype=float)
fig, ax = plt.subplots(figsize=(10, 5))
ax.stackplot(list(single_indexer_name_df.epoch), total.values(), labels = total.keys(), alpha = 0.8)
ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1.0))
ax.set_xlabel('Epoch')
ax.set_ylabel('Tokens')
ax.set_title('Delegated Tokens to Indexer vs Curated Tokens on Subgraphs Indexer is In');

Here we see that there is not much of an effect on delegation as curation events happen, as the amount of curation with respect to delegation is very small. 